In [1]:
import pandas as pd
import numpy as np
import nltk
import nltk.sentiment.util
import matplotlib.pyplot as plt
import pprint
import re
import gensim
import gensim.corpora as corpora
import os
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords

In [39]:
data = pd.read_csv('../../amazon_reviews_us_Grocery_v1_00_sentiment.csv')


In [86]:
df = data

In [87]:
df['health_hazard'] = np.zeros(df.shape[0])

In [88]:
#Only 1-Star Reviews.
#
#
#
#
#
#
#
df = df.query('star_rating < 4', engine='python')

In [89]:
df['health_hazard'].loc[df['review_body'].str.contains(" diarrhea", na=False)] = 1
df['health_hazard'].loc[df['review_body'].str.contains(" rotten", na=False)] = 1
df['health_hazard'].loc[df['review_body'].str.contains(" laxative", na=False)] = 1
df['health_hazard'].loc[df['review_body'].str.contains(" moldy", na=False)] = 1
df['health_hazard'].loc[df['review_body'].str.contains(" sick", na=False)] = 1
df['health_hazard'].loc[df['review_body'].str.contains(" ill", na=False)] = 1
df['health_hazard'].loc[df['review_body'].str.contains(" unwell", na=False)] = 1

C:\Users\MattW\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
<ipython-input-89-455cb6774c80>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['health_hazard'].loc[df['review_body'].str.contains(" diarrhea", na=False)] = 1
<ipython-input-89-455cb6774c80>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['health_hazard'].loc[df['revie

In [91]:
df.query('health_hazard == 1', engine='python')#.shape[0]

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,polarity,subjectivity,health_hazard
128,US,22581725,R1M058RU3W5Q71,B00XIGKXOG,8665580,Healthy Choice Juices - 5 Day Detox - Cold Pre...,Grocery,3,0,1,N,Y,good for a cleanse but not for taste.,This was an ok cleanse. A few of the favors w...,2015-08-31,0.038571,0.419048,1.0
301,US,15134590,R2OK571OMZ0ZDK,B004FEGLO6,359679189,"Mott's 100% Original Apple Juice, 6.75 fl oz b...",Grocery,1,2,2,N,Y,I had purchased this product before and been v...,I had to throw away one 8 pack of this juice b...,2015-08-31,0.250000,0.629630,1.0
725,US,53093920,R2RS3XC4TY4J3V,B0087EO21Y,975035888,Frontier Soups Hearty Meals Illinois Prairie C...,Grocery,2,1,1,N,N,I think i will stick to making my own---thyme ...,"First of all,this did not look anything like t...",2015-08-31,0.105093,0.394769,1.0
847,US,33974316,R1WM66IPQMCCUP,B000FKQDAQ,431762188,Life Savers 5 Flavors Sugarfree Hard Candy Bag...,Grocery,3,15,18,N,Y,"Delicious, but causes severe stomach pain and ...","Please read! These taste fantastic, however t...",2015-08-31,0.114624,0.380357,1.0
1450,US,50894327,R2PMJ72CS8IZYQ,B001EQ5AVI,188948420,"Hormel Compleats Roast Beef & Mash Potatoes, 9...",Grocery,2,0,1,N,N,"I might have gotten a bad carton, but it was m...",I love every &#34;Compleats&#34; flavor that I...,2015-08-31,0.122042,0.527489,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2400112,US,26613395,R1ALBDNXLKMNN5,B0005ZZSSE,185316897,"Little Debbie Honey Buns, 10.5 oz",Grocery,3,0,6,N,N,Its a bit too sugary but its ok,I think Honey Buns are too sugary because it m...,2005-07-14,-0.107143,0.678571,1.0
2400691,US,52195402,REFE39RI473W7,B0002BTD14,747324490,1 Case of Candy Cigarettes,Grocery,1,0,0,N,N,WARNING - Product encourages role modeling dea...,If you MUST give a child a candy containing su...,2005-02-19,0.017059,0.364150,1.0
2400751,US,51867752,R39RIQQ6ZFUFQ3,B0000D15YQ,454960017,Bitter Red,Grocery,1,10,16,N,N,gah,This made me wish I was born without taste bud...,2005-02-03,0.031250,0.230208,1.0
2401437,US,19170524,R3NR6ZHFB1BTXE,B0000CCZU2,989403560,"Teething Biscuits, Maple, Wheat/Soy/Dairy Free...",Grocery,1,12,21,N,N,Incomplete Ingredient List,I was very excited when I found these biscuits...,2004-06-21,0.028750,0.290833,1.0


In [93]:
df.query('review_id.str.contains("R1ALBDNXLKMNN5", na=False)', engine='python')['review_body'].iloc[0]

'I think Honey Buns are too sugary because it made me sick when iate this.my suggestion is eating a healthy apple or a fruit.I dont recommend Honey Buns for a snack.So please use your money and buy something healthier.'